In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from pandas.tseries.offsets import DateOffset
from tqdm import tqdm


In [10]:
# Read csv, only use data start from Feb, every row is a independent training set
data = pd.read_csv('./data/confirmed_timeseries.csv')

df = data.drop(['country', 'province', 'lat', 'long'], axis=1)
df = df.fillna(axis='index', method='ffill')
df.iloc[252].fillna()
# for i in range(22, 32):
#     df = df.drop('2020-01-{}'.format(i), axis=1)
# df.sum(axis=0).values
# num_day_pred = 7
# model = Sequential()
# model.add(LSTM(200, activation='relu', input_shape=(num_day_pred, 1)))
# model.add(Dropout(0.25))
# model.add(Dense(1))
# model.compile(loss='mse', optimizer='adam')


# date = df.columns.values.reshape(-1,1)
# for i in rqdm((len(df))):
#     arr = df.iloc[i].values
#     cur_df = pd.DataFrame(np.concatenate((date, arr.reshape(-1,1)), axis=1), columns=['date', 'val'])
#     cur_df.date = pd.to_datetime(cur_df.date)
#     cur_df = cur_df.set_index('date')

#     scaler = MinMaxScaler()
#     train = scaler.fit_transform(cur_df)
#     generator = TimeseriesGenerator(train, train, length=num_day_pred, batch_size=2)

#     model.fit_generator(generator, epochs=10)


2020-01-22       0.0
2020-01-23       0.0
2020-01-24       0.0
2020-01-25       1.0
2020-01-26       1.0
2020-01-27       1.0
2020-01-28       1.0
2020-01-29       1.0
2020-01-30       1.0
2020-01-31       2.0
2020-02-01       NaN
2020-02-02       NaN
2020-02-03       NaN
2020-02-04       NaN
2020-02-05       NaN
2020-02-06       NaN
2020-02-07       NaN
2020-02-08       NaN
2020-02-09       NaN
2020-02-10       NaN
2020-02-11       NaN
2020-02-12       NaN
2020-02-13       NaN
2020-02-14       NaN
2020-02-15       NaN
2020-02-16       NaN
2020-02-17       NaN
2020-02-18       NaN
2020-02-19       NaN
2020-02-20       NaN
               ...  
2020-02-29       NaN
2020-03-01       NaN
2020-03-02       NaN
2020-03-03       NaN
2020-03-04       NaN
2020-03-05       NaN
2020-03-06       NaN
2020-03-07       NaN
2020-03-08       NaN
2020-03-09       NaN
2020-03-10      12.0
2020-03-11      25.0
2020-03-12      32.0
2020-03-13      46.0
2020-03-14      64.0
2020-03-15      93.0
2020-03-16   

In [ ]:
df_worldwide = pd.DataFrame(np.concatenate((date, df.sum(axis=0).values.reshape(-1,1)), axis=1), columns=['date', 'val'])
df_worldwide.date = pd.to_datetime(df_worldwide.date)
df_worldwide = df_worldwide.set_index('date').astype('int')
scaler = MinMaxScaler()
train = scaler.fit_transform(df_worldwide)
generator = TimeseriesGenerator(train, train, length=num_day_pred, batch_size=2)

model.fit_generator(generator, epochs=100)



In [ ]:
preds = []
batch = train[-num_day_pred:].reshape((1, num_day_pred, 1))
for i in range(num_day_pred):
    preds.append(model.predict(batch)[0])
    batch = np.append(batch[:, 1:, :],[[preds[i]]], axis=1)

add_dates = [df_worldwide.index[-1] + DateOffset(days=x) for x in range(num_day_pred+1)]
future = pd.DataFrame(index=add_dates, columns=['val'])

df_pred = pd.DataFrame(scaler.inverse_transform(preds), index=future[-num_day_pred:].index, columns=['pred'])
df_pred.pred = pd.to_numeric(df_pred.pred).astype('int')
df_final = pd.concat((df_worldwide, df_pred), axis=1)
df_final

In [ ]:
plt.plot(df_worldwide, color='g')
plt.plot(df_pred, color='r')
plt.legend(loc='best')
plt.show()